# 🤖 Chatbot ARE (Association Robotique ENSI)
A Google Colab project to create a chatbot capable of answering questions related to ARE documents.

## 📦 Step 1: Installation des dépendances

In [1]:
!pip install --upgrade --force-reinstall pymupdf
#!pip uninstall -y torch torchvision transformers
!pip install -U langchain_huggingface

!pip install -q langchain chromadb huggingface_hub sentence-transformers langchain-community
!pip install PyMuPDF # for reading PDFs with Python
!pip install chromadb

!pip install groq

!pip install langchain
!pip install langchain_chroma

!pip install langchain_core

!pip install langchain_huggingface
!pip install sentence-transformers # for embedding models
!pip install torch torchvision transformers --upgrade --quiet
!pip install -U sentence-transformers
!pip install tqdm # for progress bars
!pip install transformers accelerate
from google.colab import userdata
from groq import Groq
from langchain.chains import RetrievalQA
from langchain.llms.base import LLM
from langchain.prompts import PromptTemplate
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from spacy.lang.fr import French
from tqdm.auto import tqdm
from typing import List, Optional
from uuid import uuid4
import fitz  # PyMuPDF
import os
import os
import pandas as pd
import random
import re
import requests


#!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

## 📄 Step 2: Chargement du document PDF

## 1. Document/Text Processing and Embedding Creation


In [2]:
pdf_path = "are_data_test.pdf"


if not os.path.exists(pdf_path):
  print("File doesn't exist !!!")

else:
  print(f"File {pdf_path} exists.")

File are_data_test.pdf exists.


## 🧹 Step 3: Nettoyage et formatage du texte

In [3]:
def text_formatter(text: str) -> str:
    """Nettoie le texte en supprimant les sauts de ligne et les espaces inutiles."""
    cleaned_text = " ".join(text.split())  # Supprime les multiples espaces et \n
    return cleaned_text

def count_sentences(text: str) -> int:
    """Compte approximativement le nombre de phrases en utilisant une segmentation plus robuste."""

    sentences = re.split(r'(?<=[.!?]) +', text)  # Sépare les phrases sur les signes de ponctuation suivis d'un espace
    return len(sentences)

def open_and_read_pdf(pdf_path: str) -> list[dict]:
    """Lit un PDF et stocke le contenu de chaque page dans une liste."""

    if not os.path.exists(pdf_path):
        raise FileNotFoundError(f"Le fichier '{pdf_path}' n'existe pas.")

    try:
        doc = fitz.open(pdf_path)
    except Exception as e:
        raise RuntimeError(f"Erreur lors de l'ouverture du PDF : {e}")

    pages_content = []

    for page_index, page_content in tqdm(enumerate(doc), total=len(doc), desc="Lecture du PDF"):
        text = page_content.get_text()
        if not text.strip():  # Ignore empty pages
            continue

        text = text_formatter(text)
        word_count = len(text.split())

        pages_content.append({
            "page_number": page_index + 1,
            "page_char_count": len(text),
            "page_word_count": word_count,
            "page_sentence_count": count_sentences(text),
            "page_token_count": len(text) / 4,  # Approximation see : https://platform.openai.com/tokenizer
            "text": text
        })

    return pages_content

In [4]:
pages_content = open_and_read_pdf( pdf_path = pdf_path)

random.sample(pages_content, k = 2)

Lecture du PDF:   0%|          | 0/5 [00:00<?, ?it/s]

[{'page_number': 4,
  'page_char_count': 3570,
  'page_word_count': 540,
  'page_sentence_count': 24,
  'page_token_count': 892.5,
  'text': "Participation dans les compétitions de robotique des autres établissements universitaires : Tous les membres de l’ARE peuvent participer dans les compétitions de robotique des autres établissements universitaires. Le nombre d’équipes participant dans chaque compétition sera fixé par le bureau exécutif selon plusieurs facteurs y compris la disponibilité de matériels. Chaque équipe, ou bien personne, souhaitant participer dans les compétitions de robotique externe doit remplir les formulaires publiées par le bureau exécutif sur le groupe de l’association pour garantir un bon déroulement de cette participation. Si une équipe/personne souhaite participer dans une compétition qui n’a pas été publiée sur le groupe de l’association, elle doit parler au directeur technique pour discuter du processus à suivre. L'empreinte de matériel de l’association pour

In [5]:
df = pd.DataFrame(pages_content)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count,page_token_count,text
0,1,1833,262,13,458.25,Règlement intérieur de l’Association Robotique...
1,2,2779,410,19,694.75,Articles : Catégories des membres de l’associa...
2,3,3628,557,25,907.00,Experts : Ce sont les membres qui ont choisi d...
3,4,3570,540,24,892.50,Participation dans les compétitions de robotiq...
4,5,736,114,4,184.00,Carte membre : Toute personne possédant une ca...


## ✂️ Step 4: Segmentation des phrases

In [6]:
nlp = French()
nlp.add_pipe("sentencizer")
doc = nlp(" bojour à tous . seconde phrase . la 3eme et la dernier phrase .")
list(doc.sents)

[ bojour à tous ., seconde phrase ., la 3eme et la dernier phrase .]

In [7]:
for page_dict in tqdm(pages_content) :
  page_dict["sentences"] = list( nlp(page_dict["text"]).sents)
  page_dict["page_sentences_count"] = len( page_dict["sentences"] )

  0%|          | 0/5 [00:00<?, ?it/s]

In [8]:
random.sample(pages_content, k = 2)

[{'page_number': 5,
  'page_char_count': 736,
  'page_word_count': 114,
  'page_sentence_count': 4,
  'page_token_count': 184.0,
  'text': 'Carte membre : Toute personne possédant une carte membre aura la priorité de participer dans toutes nos activités, d’avoir des remises exceptionnels sur les formations payantes tout en considérant les offres inclut dans cette carte et qu’elle vous fera y bénéficier. Le prix de la carte membre est fixé par le trésorier de l’association selon la disponibilité des offres incluses dans cette dernière. Local : L’entrée au local de l’association est réservée seulement aux membres de l’Association Robotique ENSI. Les membres de l’association s’engagent à se conformer aux règles et usages de local utilisé par l’association, telles que les consignes d’accès et l’utilisation des équipements, et à veiller à la bonne occupation des lieux.',
  'sentences': [Carte membre : Toute personne possédant une carte membre aura la priorité de participer dans toutes nos a

## 📊 Step 5: Aperçu du DataFrame

In [9]:
df = pd.DataFrame(pages_content)
df.describe()

,page_number,page_char_count,page_word_count,page_sentence_count,page_token_count,page_sentences_count
count,5.000000,5.00000,5.00000,5.000000,5.000000,5.000000
mean,3.000000,2509.20000,376.60000,17.000000,627.300000,17.200000
std,1.581139,1229.96817,188.71354,8.689074,307.492043,8.927486
min,1.000000,736.00000,114.00000,4.000000,184.000000,4.000000
25%,2.000000,1833.00000,262.00000,13.000000,458.250000,13.000000
50%,3.000000,2779.00000,410.00000,19.000000,694.750000,19.000000
75%,4.000000,3570.00000,540.00000,24.000000,892.500000,24.000000
max,5.000000,3628.00000,557.00000,25.000000,907.000000,26.000000


## 🧠 Step 6: Création des embeddings avec Sentence Transformers

## **Break our pages of sentences into groups of 3**
Why do we do this?

* Easier to manage similar sized chunks of text.
* Don't overload the embedding models capacity for tokens(respect to context window of our embedding model).



In [10]:
num_sentences_per_list = 3

def split_list_into_sublists(input_list : list[str], num_elements_per : int) -> list[list[str]] :
  return [input_list[i:i + num_elements_per] for i in range(0, len(input_list), num_elements_per)]

In [11]:
for item in tqdm(pages_content):
  item["sentences_chunks"] = split_list_into_sublists(item["sentences"], num_sentences_per_list)
  item["num_sentences_chunks"] = len(item["sentences_chunks"])

  0%|          | 0/5 [00:00<?, ?it/s]

In [12]:
df = pd.DataFrame(pages_content)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count,page_token_count,text,sentences,page_sentences_count,sentences_chunks,num_sentences_chunks
0,1,1833,262,13,458.25,Règlement intérieur de l’Association Robotique...,"[(Règlement, intérieur, de, l’, Association, R...",13,"[[(Règlement, intérieur, de, l’, Association, ...",5
1,2,2779,410,19,694.75,Articles : Catégories des membres de l’associa...,"[(Articles, :, Catégories, des, membres, de, l...",19,"[[(Articles, :, Catégories, des, membres, de, ...",7
2,3,3628,557,25,907.00,Experts : Ce sont les membres qui ont choisi d...,"[(Experts, :, Ce, sont, les, membres, qui, ont...",26,"[[(Experts, :, Ce, sont, les, membres, qui, on...",9
3,4,3570,540,24,892.50,Participation dans les compétitions de robotiq...,"[(Participation, dans, les, compétitions, de, ...",24,"[[(Participation, dans, les, compétitions, de,...",8
4,5,736,114,4,184.00,Carte membre : Toute personne possédant une ca...,"[(Carte, membre, :, Toute, personne, possédant...",4,"[[(Carte, membre, :, Toute, personne, possédan...",2


## 🗂️ Step 7: Indexation des embeddings avec ChromaDB

In [13]:
random.sample(pages_content, k = 2)

[{'page_number': 2,
  'page_char_count': 2779,
  'page_word_count': 410,
  'page_sentence_count': 19,
  'page_token_count': 694.75,
  'text': 'Articles : Catégories des membres de l’association : L’Association Robotique ENSI se compose de : - Bureau exécutif - Staff technique - Comité d’organisation - Comité de conseil - Anciens membres - Chefs d’opérations - Experts - Nouveaux membres Bureau exécutif : C’est l’entité dirigeante de l’association. Les membres du bureau exécutif veillent à la bonne gestion de l’association, au respect des règles ainsi qu’aux mécanismes statutaires de l’association, à l’organisation des évènements, formations, à l’encadrement des membres de l’ARE lors de leurs participations dans les compétitions externes. Le bureau exécutif actuel se compose de : Président. Vice-Président Secrétaire générale et directrice des ressources humaines Trésorier Directrice commerciale Directeur de communauté Directrice de communication Directeur technique La composition du bure

In [14]:
df.describe()

,page_number,page_char_count,page_word_count,page_sentence_count,page_token_count,page_sentences_count,num_sentences_chunks
count,5.000000,5.00000,5.00000,5.000000,5.000000,5.000000,5.000000
mean,3.000000,2509.20000,376.60000,17.000000,627.300000,17.200000,6.200000
std,1.581139,1229.96817,188.71354,8.689074,307.492043,8.927486,2.774887
min,1.000000,736.00000,114.00000,4.000000,184.000000,4.000000,2.000000
25%,2.000000,1833.00000,262.00000,13.000000,458.250000,13.000000,5.000000
50%,3.000000,2779.00000,410.00000,19.000000,694.750000,19.000000,7.000000
75%,4.000000,3570.00000,540.00000,24.000000,892.500000,24.000000,8.000000
max,5.000000,3628.00000,557.00000,25.000000,907.000000,26.000000,9.000000


In [15]:
print(pages_content[1]["sentences_chunks"])
print (len(pages_content[1]["sentences_chunks"]))

[[Articles : Catégories des membres de l’association : L’Association Robotique ENSI se compose de : - Bureau exécutif - Staff technique - Comité d’organisation - Comité de conseil - Anciens membres - Chefs d’opérations - Experts - Nouveaux membres Bureau exécutif : C’est l’entité dirigeante de l’association., Les membres du bureau exécutif veillent à la bonne gestion de l’association, au respect des règles ainsi qu’aux mécanismes statutaires de l’association, à l’organisation des évènements, formations, à l’encadrement des membres de l’ARE lors de leurs participations dans les compétitions externes., Le bureau exécutif actuel se compose de : Président.], [Vice-Président Secrétaire générale et directrice des ressources humaines Trésorier Directrice commerciale Directeur de communauté Directrice de communication Directeur technique La composition du bureau exécutif peut être modifiée selon le besoin évoqué., Staff technique : C’est l’entité chargée d’assurer le travail sur le plan techni

# Splitting each chunk of sentence separately
 pages_sentence_chunks is a list of dict that containes a single  chunk of sentences and relative informations




In [16]:
pages_sentence_chunks = []
for item in tqdm(pages_content):
    for sentence_chunk in item["sentences_chunks"]:
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]

        joined_sentence_chunk = "".join([span.text for span in sentence_chunk]).replace("  ", " ").strip()
        joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk) # ".A" -> ". A"
        chunk_dict["sentence_chunk"] = joined_sentence_chunk

        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4 # 1 token = ~4 characters

        pages_sentence_chunks.append(chunk_dict)

  0%|          | 0/5 [00:00<?, ?it/s]

In [17]:
random.sample(pages_sentence_chunks, k=1)

[{'page_number': 5,
  'sentence_chunk': 'Les membres de l’association s’engagent à se conformer aux règles et usages de local utilisé par l’association, telles que les consignes d’accès et l’utilisation des équipements, et à veiller à la bonne occupation des lieux.',
  'chunk_char_count': 224,
  'chunk_word_count': 35,
  'chunk_token_count': 56.0}]

## 🧵 Step 8: Configuration de la chaîne de questions-réponses (RetrievalQA)

In [18]:
min_token_length = 30
df = pd.DataFrame(pages_sentence_chunks)

filtered_df = df[df["chunk_token_count"] <= min_token_length]
if filtered_df.empty:
    print(f"No chunks found with token count less than or equal to {min_token_length}")
else:
    # Get the number of rows in filtered_df
    num_rows = len(filtered_df)

    # Ensure sample size is not larger than the number of rows
    sample_size = min(3, num_rows)

    for row in filtered_df.sample(sample_size).iterrows(): # Use sample_size instead of 3
        print(f'Chunk token count: {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_chunk"]}')

Chunk token count: 24.0 | Text: Le chef d’opération est garant de l’organisation et du pilotage des travaux jusqu’à leur terme .


* #  Embedding our text
Great work so far, Now that we’ve successfully split the text into sentence chunks, **our next step is to transform these chunks into numerical representations**, through a process known as embedding.

 This step is crucial because it allows us to convert the textual data into a format that machine learning models can understand and process. Essentially, we’ll use a pre-trained embedding model (`all-mpnet-base-v2` is the most recommended see : https://sbert.net/docs/sentence_transformer/pretrained_models.html) to map each chunk of text into a high-dimensional vector space.

  Steps :
* Embed the text chunks using the chosen model.
* Save the embeddings for easy access.
* Experiment with similarity searches to retrieve semantically relevant data.



In [19]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

sentences = [
    "That is a happy person",
    "That is a happy dog",
    "That is a very happy person",
    "Today is a sunny day"
]
embeddings = embedding_model.embed_documents(sentences)

similarities = embedding_model.embed_query("That is a happy person")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [20]:
print(len(embeddings))

4


In [21]:
print(similarities)

[-0.020344296470284462, -0.010426380671560764, 0.00858661811798811, 0.02715197764337063, 0.04989389330148697, 0.0012354827485978603, -0.04918303340673447, -0.004081337712705135, 0.049975182861089706, 0.04413984343409538, -0.04122484102845192, 0.05667446553707123, -0.05389859527349472, -0.024612849578261375, 0.01963088847696781, -0.02573440410196781, 0.010839688591659069, 0.07730206102132797, -0.03512104973196983, 0.02399689331650734, 0.004788292571902275, 0.054373033344745636, 0.023779993876814842, 0.0016143601387739182, -0.011864547617733479, 0.0031496603041887283, 0.006577759515494108, 0.021422166377305984, 0.0005437638610601425, -0.030256768688559532, 0.00734828831627965, -0.03385508060455322, -0.05172358825802803, 0.017943359911441803, 1.5886548681010026e-06, 0.016542738303542137, 0.035514041781425476, 0.009180167689919472, -0.0017198655987158418, 0.006751285400241613, 0.019232138991355896, -0.03616679459810257, -0.04721591994166374, -0.01712876185774803, 0.014357483945786953, 0.04

In [22]:
vector_store = Chroma(
    collection_name="are_collection",
    embedding_function=embedding_model,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

## 💬 Step 9: Interface de chat utilisateur

# Adding items to vector store

In [23]:
doc1 = Document(page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose", metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},id = "1")
doc2 = Document(page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...", metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},id = "2")

vector_store.add_documents(documents=[doc1,doc2])

['1', '2']

In [24]:
vector_store.delete(ids=[str(1),str(2)])

In [25]:
doc_list = [Document(page_content = item["sentence_chunk"]) for item in pages_sentence_chunks]

In [26]:
print (doc_list[2])

page_content='Toute nouvelle modification établie au règlement doit être communiquée aux membres de l’ARE. Introduction : L’association Robotique ENSI est une association à but non lucratif fondée en 2006 au sein de l’Ecole nationale des Sciences de l’Informatique (ENSI). Elle englobe plus d’une centaine de membres actifs.'


In [27]:
uuids = [str(uuid4()) for _ in range(len(doc_list))] # creating id for each document

vector_store.add_documents(documents = doc_list, ids = uuids)

['6b3205e9-b03e-4122-bc4e-daaa7949ccc5',
 '35251e4e-a682-452c-b862-e20fd14cc925',
 '8be71318-7faf-40fc-9ba4-b1bcfde08b14',
 'd3f8c40d-37a1-47ea-b15e-8e33eaa85767',
 'bed5dcf2-4cac-4ab1-927f-17a8ed9377ca',
 '2b34a766-8917-4ab1-bbe9-923fd48526fe',
 '3d47842c-4d35-4b72-9b59-40c889b3b586',
 'e41521c4-6b51-48e5-bbb0-6d3a0bb61e59',
 '88c4e029-3d71-4b93-a867-4d46d7966726',
 'dc99b3b8-01cb-4b3f-bbda-683ed35e3e6a',
 'd8565a43-1f96-4074-b259-592823e481ea',
 '7f42047f-1345-494a-b359-be8988297d7a',
 '15634334-3b4a-4409-8d0c-75f23c4d7506',
 'f2a544c7-46c3-4466-9e52-a71366e42555',
 '85ec7d7a-cb39-4fe7-818f-348b06eff7a3',
 'a691255a-aec7-4ce3-9d09-e836119cebbb',
 '770e8624-1010-4576-9719-c77259495291',
 '0493222f-1f9d-4fe1-8114-53440a0bfe62',
 '5d683493-ffed-45f1-a54c-1df637a6b52f',
 '757e70bc-4d9e-4599-b714-7ff40cbfe2a4',
 'c9ad2e8f-04c8-49ae-91ad-900345b47ce2',
 'fb820e73-e098-4617-9017-2e2e15748bbc',
 'af1d713c-9f85-4fe4-a2d0-28c42e111e1d',
 'cf374d92-c1a8-44e7-880f-8bc4e6075fc9',
 'bedceabf-21cf-

In [28]:
results = vector_store.similarity_search(
    " la composition du staff technique",
    k=2
)
for res in results:
    print(f"* {res.page_content} ")

* Le staff technique n’intervient pas dans la prise des décisions non techniques. Le staff technique est composé de : Responsables formations. Responsables compétitions. 
* Vice-Président Secrétaire générale et directrice des ressources humaines Trésorier Directrice commerciale Directeur de communauté Directrice de communication Directeur technique La composition du bureau exécutif peut être modifiée selon le besoin évoqué. Staff technique : C’est l’entité chargée d’assurer le travail sur le plan technique au sein de l’association en collaborant avec le directeur technique. Elle est choisie par le bureau exécutif du mandat précédent. 


In [29]:
class GroqLLM(LLM):
    model: str
    api_key: str

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        client = Groq(api_key=self.api_key)
        messages = [{"role": "user", "content": prompt}]
        response = client.chat.completions.create(
            messages=messages,
            model=self.model
        )
        return response.choices[0].message.content

    @property
    def _llm_type(self) -> str:
        return "groq"




In [42]:
llm = GroqLLM(
    model="deepseek-r1-distill-llama-70b",
    api_key=userdata.get('groq')
)

In [43]:
prompt_template = PromptTemplate.from_template("""
You are Robotique-Info, a knowledgeable and long-standing member of the 'ARE' (Association Robotique ENSI).
Answer the user's question truthfully and only based on the provided context.
If a user asks a question that is clearly outside the scope of the "Association Robotique ," such as questions about unrelated topics, general knowledge, or other organizations, you must politely decline to answer. In such cases, respond with a phrase like:

* "This question is outside the scope of the Association Robotique."
* "I can only answer questions related to the Association Robotique."
* "My knowledge is limited to information about the Association Robotique."
* "I'm sorry, but I cannot provide information on that topic as it's not related to the Association Robotique."

Context:
{context}

Question:
{question}

Answer:
""")

In [44]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 8})

In [45]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt_template}
)

In [46]:
query = "How to contact ARE?"
response = qa_chain.run(query)

def remove_think_tags(text):
    return re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL)

clean_response = remove_think_tags(response)

print(clean_response)



You can contact the Association Robotique ENSI (ARE) through the following means:

- **Email:** association.robotique@ensi-uma.tn  
- **Phone:** +216 40 485 430  
- **Website:** www.association-robotique-ensi.tn  
- **Address:** Campus universitaire de la Manouba, Manouba 2010  

Feel free to reach out using any of these methods for more information or inquiries.


In [47]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 85.3 MB/s eta 0:00:00


In [48]:
import gradio as gr

# Example function (replace this with your actual RAG pipeline)
def generate_answer(query):
    response = qa_chain.run(query)
    clean_response = remove_think_tags(response)
    return clean_response

custom_css = """
body {
    background-image: url('https://your-image-url.com/background.jpg');
    background-size: cover;
    background-repeat: no-repeat;
    background-attachment: fixed;
}
"""

# Build the Gradio UI
iface = gr.Interface(
    fn=generate_answer,
    inputs=gr.Textbox(lines=2, placeholder="Ask a question..."),
    outputs=gr.Textbox(),
    title="ARE CHATBOT",
    #description="You are Robotique-Info, a knowledgeable and long-standing member of the 'ARE' (Association Robotique ENSI).Answer the user's question truthfully and only based on the provided context..",
    css = custom_css
)

# Launch the app
iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a1cd735776d523ee0f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
